In [1]:
# &biw=1309
# &bih=717
# &tbm=nws
# &lr=lang_en
# &hl=en
# &tbs=cdr:1,cd_min:1/2/2020,cd_max:1/2/2020
# &num=20

In [2]:
# GOOG, AMZN, MSFT, JPM, WMT, NKE, DIS

In [3]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import quote, parse_qs, urlparse, urlencode
import time

USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
headers={"user-agent" : USER_AGENT}
target='AAPL' # target symbol
keyword = quote(target.encode('utf8'))

total_search = {}
traind_date_list = pd.date_range(start="2009-11-31",end="2019-12-31").to_list()
start_time = time.time()

for i in traind_date_list:
    time.sleep(0.1) # prevent from anti crawling
    target_date = i.strftime("%m/%d/%Y")
    if(i.month == 12 and i.day==31){
        print("Finish Process Year ", i.year)
    }
    # google query parameters
    target_param = {
        "tbm": "nws", # search news
        "hl": "en",
        "lr": "lang_en", # in English
        "q": keyword, # search keyword
        "oq": keyword, # search keyword
        "dcr": "0",
        "source": "lnt",
        "num": 5, # top 5 news
        "tbs": "cdr:1,cd_min:"+target_date+",cd_max:"+target_date, # specific date
    }
    url = "https://www.google.com.tw/search?" + urlencode(target_param)
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        content = res.content
        soup = BeautifulSoup(content, "html.parser")
        search_list = []
        items = soup.findAll("div", {"class": "g"})
        if items:
            for index, item in enumerate(items):
                    # title
                    news_title = item.find("h3", {"class": "r"}).find("a").text
                    # url
                    href = item.find("h3", {"class": "r"}).find("a").get("href")
                    news_link = href
                    # content
                    news_text = item.find("div", {"class": "st"}).text
                    # source
                    news_source = item.find("h3", {"class": "r"}).findNext('div').text.split('-')
                    news_from = news_source[0]
                    time_created = str(news_source[1])
                    # add item into dict, and later store them in JSON
                    search_list.append({
                        "news_title": news_title,
                        "news_link": news_link,
                        "news_text": news_text,
                        "news_from": news_from,
                        "time_created": time_created
                    })
            total_search[target_date] = search_list
    else:
        print('error at '+str(i))

SyntaxError: invalid syntax (<ipython-input-3-927d7f373ae2>, line 20)

In [169]:
import json
with open('AAPL_news_09-19.json', 'w') as fp:
    json.dump(total_search, fp)

In [1]:
import json

with open('AAPL_news_09-19.json') as json_file:
    data = json.load(json_file)

# Function Wise

In [60]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import quote, parse_qs, urlparse, urlencode
import time

In [62]:
def crawl_news(target="AAPL"):
#     USER_AGENT = "Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1"
#     headers={"user-agent" : USER_AGENT}
    keyword = quote(target.encode('utf8'))

    total_search = {}
    traind_date_list = pd.date_range(start="2009-11-30",end="2019-12-31").to_list()
    start_time = time.time()
    header = Headers(
        headers=True  # generate misc headers
    )
    headers = header.generate()

    for i in traind_date_list:
        time.sleep(0.5) # prevent from anti crawling
        target_date = i.strftime("%m/%d/%Y")
        if(i.month == 12 and i.day==31):
            print("Finish Process ", target, "News in Year ", i.year)
        # google query parameters
        target_param = {
            "tbm": "nws", # search news
            "hl": "en",
            "lr": "lang_en", # in English
            "q": keyword, # search keyword
            "dcr": "0",
            "source": "lnt",
            "num": 5, # top 5 news
            "tbs": "cdr:1,cd_min:"+target_date+",cd_max:"+target_date, # specific date
        }
        url = "https://www.google.com/search?" + urlencode(target_param)
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            content = res.content
            soup = BeautifulSoup(content, "html.parser")
            search_list = []
            items = soup.findAll("div", {"class": "g"})
            if items:
                for index, item in enumerate(items):
                        # title
                        news_title = item.find("h3", {"class": "r"}).find("a").text
                        # url
                        href = item.find("h3", {"class": "r"}).find("a").get("href")
                        news_link = href
                        # content
                        news_text = item.find("div", {"class": "st"}).text
                        # source
                        news_source = item.find("h3", {"class": "r"}).findNext('div').text.split('-')
                        news_from = news_source[0]
                        time_created = str(news_source[1])
                        # add item into dict, and later store them in JSON
                        search_list.append({
                            "news_title": news_title,
                            "news_link": news_link,
                            "news_text": news_text,
                            "news_from": news_from,
                            "time_created": time_created
                        })
                total_search[target_date] = search_list
        else:
            print('error at '+str(i))
            break
    return total_search

In [63]:
import json
target_list = ["GOOG", "AMZN", "MSFT", "JPM", "WMT", "NKE", "DIS"]
target = "WMT"
for i in target_list:
    news = crawl_news(i)
    with open('./data/'+i+'/news_09-19.json', 'w') as fp:
        json.dump(news, fp)

Finish Process  GOOG News in Year  2009


KeyboardInterrupt: 

In [164]:
from urllib.parse import urlencode
kk = "12/30/2008"
target_date = {
    "tbm": "nws",
    "hl": "en",
    "lr": "lang_en",
    "q": keyword,
    "oq": keyword,
    "dcr": "0",
    "source": "lnt",
    "tbs": "cdr:1,cd_min:"+kk+",cd_max:"+kk,
}
print("https://www.google.com.tw/search?"+urlencode(target_date))

https://www.google.com.tw/search?tbm=nws&hl=en&lr=lang_en&q=AAPL&oq=AAPL&dcr=0&source=lnt&tbs=cdr%3A1%2Ccd_min%3A12%2F30%2F2008%2Ccd_max%3A12%2F30%2F2008


In [13]:
import pandas as pd
from datetime import datetime
from urllib.parse import quote, parse_qs, urlencode
import time
traind_date_list = pd.date_range(start="2008-12-30",end="2009-01-15").to_list()
for i in traind_date_list:
    after_format = i.strftime("%m/%d/%Y")
    print(i.month == 12 and i.day==31)

False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [113]:
items = soup.findAll("div", {"class": "g"})

In [125]:
search_list = []
for index, item in enumerate(items):
        # title
        news_title = item.find("h3", {"class": "r"}).find("a").text
        # url
        href = item.find("h3", {"class": "r"}).find("a").get("href")
        news_link = href
        

        # content
        news_text = item.find("div", {"class": "st"}).text

        # source
        news_source = item.find("h3", {"class": "r"}).findNext('div').text.split('-')
        news_from = news_source[0]
        time_created = str(news_source[1])

        # add item into json object
        search_list.append({
            "news_title": news_title,
            "news_link": news_link,
            "news_text": news_text,
            "news_from": news_from,
            "time_created": time_created
        })